# 03 - Algorithms

In this notebook you can find simple examples that showcase how to predict the best future portfolio values according to some heuristic.

You can create your own algorithms by creating a Python module in `modules/experts/`. You have to follow the format of `modules/experts/Sharpe.py`.

In [1]:
import sys
sys.path.insert(0, '../')
%reload_ext autoreload
%autoreload 2

## Start server

The `Server` class is the interface to perform HTTPS requests to acquire data regarding instruments, like stocks.

In [2]:
import time
import logging

import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

from modules.server import Server
from modules.instruments import Portfolio

In [3]:
srv = Server()
srv.start()

# the following silences the logging output
# logging.getLogger('ibkr-algotrading').setLevel(logging.ERROR)

Once started, we can request information on any stock we want simply by accesing items in `srv`. These will be cached, so only the first access will have significant delay.

## Create Portfolio

we can now request some stocks and create a portfolio with them, adding the portfolio to the server so it knows about its existance

In [5]:
# historical data in a 1 year period in 1 day intervals
stocks = srv[['SPY', 'XOM', 'MSFT', 'AAPL', 'TSLA'], '1y', '1d']

# wealth of 100 USD
portfolio = Portfolio(wealth=100, stocks=stocks)

srv.add_portfolio(portfolio)

[2021-09-08 10:36:59,937] I T140145377912384 server.py:467: Started StockCache thread for bar "1d". 
		Next data update will be in  55380.06s.
[2021-09-08 10:37:00,012] I T140145369519680 server.py:449: Loaded "Stock("XOM", conid=13977)" from StockCache's storage.
[2021-09-08 10:37:00,033] I T140145361126976 server.py:449: Loaded "Stock("MSFT", conid=272093)" from StockCache's storage.
[2021-09-08 10:37:00,050] I T140145352734272 server.py:449: Loaded "Stock("AAPL", conid=265598)" from StockCache's storage.
[2021-09-08 10:37:00,068] I T140145344341568 server.py:449: Loaded "Stock("TSLA", conid=76792991)" from StockCache's storage.


## Experts

Experts are sub-algorithms that provide a prediction to how the wealth of the portfolio should be allocated.

We will use an expert that predicts the allocation that maximizes the sharpe ratio of the portfolio.

In [19]:
from modules import experts

# some experts may need initialisation
experts.Sharpe.init()

# predict for tomorrow
date = pd.Timestamp.today() + pd.Timedelta(days=1)
print('Predicting for date:', date)

weights = experts.Sharpe.predict(pf=portfolio, date=date, srv=srv)

[2021-09-08 20:18:08,337] I T140146814007104 Sharpe.py:69: Starting Sharpe ratio optimizer for dates: 2020-09-09 13:30:00 -> 2021-09-08 20:18:08.337229


Predicting for date: 2021-09-09 20:18:08.335920


[2021-09-08 20:18:11,000] I T140146814007104 Sharpe.py:86: sharpe:  1.89 
		weights: [0.25, 0.25, 0.25, 0.25, 0.25]
[2021-09-08 20:18:13,491] I T140146814007104 Sharpe.py:86: sharpe:  1.89 
		weights: [0.25, 0.25, 0.25, 0.25, 0.25]
[2021-09-08 20:18:15,980] I T140146814007104 Sharpe.py:86: sharpe:  1.89 
		weights: [0.25, 0.25, 0.25, 0.25, 0.25]
[2021-09-08 20:18:18,449] I T140146814007104 Sharpe.py:86: sharpe:  1.89 
		weights: [0.25, 0.25, 0.25, 0.25, 0.25]
[2021-09-08 20:18:20,894] I T140146814007104 Sharpe.py:86: sharpe:  1.89 
		weights: [0.25, 0.25, 0.25, 0.25, 0.25]
[2021-09-08 20:18:23,397] I T140146814007104 Sharpe.py:86: sharpe:  1.89 
		weights: [0.25, 0.25, 0.25, 0.25, 0.25]
[2021-09-08 20:18:25,868] I T140146814007104 Sharpe.py:86: sharpe:  1.72 
		weights: [0.26, 0.47, 0.27, 2.8e-17, 0]
[2021-09-08 20:18:28,324] I T140146814007104 Sharpe.py:86: sharpe:  1.99 
		weights: [0.25, 0.33, 0.26, 0.15, 0.15]
[2021-09-08 20:18:30,797] I T140146814007104 Sharpe.py:86: sharpe:  1.99